In [1]:
# Imports y configuración
import os
import pandas as pd
import numpy as np
from pathlib import Path
from scipy import stats

DATA_RAW = Path('/home/camilo-pc/gestion-datos/NHANES2009-2012.csv')
DATA_CLEAN_DIR = Path('/home/camilo-pc/gestion-datos/noteebook/nhanes_clean')
DATA_CLEAN_DIR.mkdir(parents=True, exist_ok=True)
DATA_CLEAN = DATA_CLEAN_DIR / 'NHANES2009-2012_sex_clean.csv'

# Columnas de interés según el CSV crudo
COLS = [
    'SurveyYr',        # año encuesta
    'ID',              # id
    'Gender',          # sexo biológico reportado
    'Age',             # edad
    'SexEver',         # ha tenido sexo alguna vez
    'SexAge',          # edad del primer acto sexual
    'SexNumPartnLife', # # parejas de por vida
    'SexNumPartYear'   # # parejas último año
]

# Diccionario de renombrado a snake_case
RENAME = {
    'SurveyYr': 'surveyyr',
    'ID': 'id',
    'Gender': 'gender',
    'Age': 'age',
    'SexEver': 'sexever',
    'SexAge': 'sexage',
    'SexNumPartnLife': 'sexnumpartnlife',
    'SexNumPartYear': 'sexnumpartnyear'
}

assert DATA_RAW.exists(), f'No existe archivo crudo: {DATA_RAW}'


PermissionError: [Errno 13] Permission denied: '/home/camilo-pc'

In [ ]:
# Carga selectiva y estandarización de nombres
raw = pd.read_csv(DATA_RAW, usecols=COLS)
raw.columns = [c.strip() for c in raw.columns]
df = raw.rename(columns=RENAME).copy()

# Tipos y codificaciones
# sexever: mapear a booleano
if df['sexever'].dtype == object:
    df['sexever'] = df['sexever'].str.strip().str.lower().map({'yes': True, 'no': False, 'y': True, 'n': False})

df['sexage'] = pd.to_numeric(df['sexage'], errors='coerce')
df['sexnumpartnlife'] = pd.to_numeric(df['sexnumpartnlife'], errors='coerce')
df['sexnumpartyear'] = pd.to_numeric(df['sexnumpartyear'], errors='coerce')
df['age'] = pd.to_numeric(df['age'], errors='coerce')

# Validación de columnas
expected = set(RENAME.values())
missing = expected.difference(df.columns)
assert not missing, f'Faltan columnas esperadas: {missing}'

df.head(3)


In [ ]:
# Limpieza básica: duplicados, filtros y correcciones obvias
# Duplicados por (surveyyr, id)
df = df.sort_values(['surveyyr','id']).drop_duplicates(subset=['surveyyr','id'], keep='first')

# Nos enfocamos en personas sexualmente activas
if 'sexever' in df:
    df = df[df['sexever'] == True].copy()

# Correcciones de valores imposibles/incorrectos
# - sexage fuera de rango plausible (<8 o >60) -> NaN
# - sexnumpartnlife/year negativos -> NaN
# - edad al primer sexo no puede exceder la edad actual
mask_invalid_age = (df['sexage'] < 8) | (df['sexage'] > 60)
df.loc[mask_invalid_age, 'sexage'] = np.nan

for col in ['sexnumpartnlife', 'sexnumpartyear']:
    df.loc[df[col] < 0, col] = np.nan

# sexage > age -> NaN (si age disponible)
if 'age' in df.columns:
    df.loc[df['sexage'] > df['age'], 'sexage'] = np.nan

# Mínima completitud para análisis: requerimos sexage presente
# y al menos uno de los contadores de parejas no nulo
df = df[(~df['sexage'].isna()) & (~(df[['sexnumpartnlife','sexnumpartyear']].isna().all(axis=1)))]

len(df)


In [ ]:
# Tratamiento de atípicos

def winsorize_series(s, lower_q=0.01, upper_q=0.99):
    s_clean = s.copy()
    q_low, q_high = s_clean.quantile([lower_q, upper_q])
    return s_clean.clip(lower=q_low, upper=q_high)

# Conteos: aplicar winsorización robusta
for col in ['sexnumpartnlife', 'sexnumpartyear']:
    if col in df.columns:
        df[col] = winsorize_series(df[col])

# sexage: no aplicar winsor puro; ya acotamos por reglas plausibles
# Crear variables transformadas para análisis opcional
for col in ['sexnumpartnlife', 'sexnumpartyear']:
    if col in df.columns:
        df[f'{col}_log1p'] = np.log1p(df[col])

df['sexage_z'] = (df['sexage'] - df['sexage'].mean()) / df['sexage'].std(ddof=0)

df.describe(include='all').T.head(12)


In [ ]:
# Columnas constantes/redundantes (en este subconjunto)
unique_counts = df.nunique(dropna=False)
constant_cols = unique_counts[unique_counts <= 1].index.tolist()
constant_cols


In [ ]:
# Guardar CSV limpio minimal para el análisis
cols_export = [
    'surveyyr','id','gender','age','sexever','sexage',
    'sexnumpartnlife','sexnumpartyear',
    'sexnumpartnlife_log1p','sexnumpartyear_log1p','sexage_z'
]
cols_export = [c for c in cols_export if c in df.columns]

# Orden estable
df_clean = df[cols_export].copy()
df_clean.to_csv(DATA_CLEAN, index=False)

DATA_CLEAN, df_clean.shape


In [ ]:
# Correlaciones
from scipy.stats import pearsonr, spearmanr

corrs = {}
if 'sexnumpartnlife' in df.columns:
    valid = df[['sexage','sexnumpartnlife']].dropna()
    r_p, p_p = pearsonr(valid['sexage'], valid['sexnumpartnlife'])
    r_s, p_s = spearmanr(valid['sexage'], valid['sexnumpartnlife'])
    corrs['sexage_vs_life'] = {'pearson_r': r_p, 'pearson_p': p_p, 'spearman_r': r_s, 'spearman_p': p_s, 'n': len(valid)}

if 'sexnumpartyear' in df.columns:
    valid = df[['sexage','sexnumpartyear']].dropna()
    r_p, p_p = pearsonr(valid['sexage'], valid['sexnumpartyear'])
    r_s, p_s = spearmanr(valid['sexage'], valid['sexnumpartyear'])
    corrs['sexage_vs_year'] = {'pearson_r': r_p, 'pearson_p': p_p, 'spearman_r': r_s, 'spearman_p': p_s, 'n': len(valid)}

corrs
